In [8]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import thinc.extra.datasets
import pandas as pd
import spacy
from spacy.util import minibatch, compounding
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from numpy import array, unique, array_equal, zeros, int8, append
max_fatures = 4000


def load_data():
    print("Loading Data...")
    PrimaryEmotion = pd.read_csv('emotion.xls.csv')
    print("Data Loaded")
    return (PrimaryEmotion['sentence'], array(PrimaryEmotion['emotion']))



def sort_to_2_emotions(sentence_list, emotion_list):
    sorted_list = []
    sorted_emo = []
    for (data, emo) in zip(sentence_list, emotion_list):
        if (emo == 'Joy' or emo == 'Love' or emo == 'Optimism' or emo == 'Awe' or emo == 'Trust'):
            sorted_list.append(data)
            sorted_emo.append("positive")
        if (emo == 'anger' or emo == 'Disgust' or emo == 'Sadness' or emo == 'Aggression' or emo == 'Contempt' or emo == 'Disapproval' or emo == 'Remorse'):
            sorted_list.append(data)
            sorted_emo.append("negative")
    return (sorted_list, sorted_emo)



def tokenize(sentences):
    print("Tokenizing...")
    tokenizer = Tokenizer(num_words=max_fatures, split=' ')
    tokenizer.fit_on_texts(sentences)
    X = tokenizer.texts_to_sequences(sentences)
    X = pad_sequences(X)
    print("Tokenization Completed")
    return X



def train(tokenized_sentences, emotion_list):
    embed_dim = 128
    lstm_out = 196
    test_percent=.20

    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = tokenized_sentences.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    
    # len(set(emotion_list)) is a hacky way of geting the number of unique elements
    # in a regualar python list (non-numpy)
    model.add(Dense(unique(emotion_list).size,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    #print(model.summary())

    Y = pd.get_dummies(emotion_list).values
    #X_train, X_test, Y_train, Y_test = train_test_split(tokened_sentences,Y, test_size = test_percent, random_state = 42)
    #print(X_train.shape,Y_train.shape)
    #print(X_test.shape,Y_test.shape)
    batch_size = 32
    return model

### MISTAKE_LIST FORMAT::
### MISTAKE_LIST [VALUES FOR WHICH MISTAKE WAS MADE][INDEX OF MISTAKES]

def test(model, tokenized_data, sentence_list, emotion_list, mistake_list):
    predictions = model.predict(tokenized_data, batch_size=32)
    error = 0
    for i in range(len(predictions)):
            pos = list(predictions[i]).index(max(predictions[i]))
            if (unique(emotion_list)[pos] != emotion_list[i]):
                mistake_list[list(unique(emotion_list)).index(emotion_list[i])][pos] += 1
    return mistake_list

    
    
#################################
#############MAIN################
#################################
#   jupyter notebook is weird   #




(sentence_list, emotion_list) = load_data()


### Comment the Below line for all 18 emotions. This sorts into "positive" and "negative"
#sentence_list, emotion_list = sort_to_2_emotions(sentence_list, emotion_list)

tokenized_data = tokenize(sentence_list)

mistake_list = zeros((18, 18), dtype=int8)



#breaks
print("Training may take a while. Please be patient")
for i in range(0,4):
    print('Starting training {} / 5 ...'.format(str(i+1)))
    testing_data_start = int((i*len(tokenized_data))/5)
    testing_data_end = int((((i+1)*len(tokenized_data))/5))
    
    testing_data = tokenized_data[testing_data_start:testing_data_end]
    testing_emo = emotion_list[testing_data_start:testing_data_end]
                       
    training_data = array(list(tokenized_data[:testing_data_start]) +  list( tokenized_data[testing_data_end:] ))
    training_emo = array(list(emotion_list[:testing_data_start]) + list(emotion_list[testing_data_end:]    ))
    model = train(training_data, training_emo)
    mistake_list = test(model, testing_data, sentence_list, emotion_list, mistake_list)
print("Training Complete!")


Loading Data...
Data Loaded
Tokenizing...
Tokenization Completed
hello 0
Training...
Epoch 1/10
2020/2020 [==============================] - 14s 7ms/step - loss: 2.6515 - acc: 0.2050
Epoch 2/10
2020/2020 [==============================] - 13s 6ms/step - loss: 2.5830 - acc: 0.2114
Epoch 3/10
2020/2020 [==============================] - 14s 7ms/step - loss: 2.5184 - acc: 0.2238
Epoch 4/10
2020/2020 [==============================] - 12s 6ms/step - loss: 2.2768 - acc: 0.3069
Epoch 5/10
2020/2020 [==============================] - 12s 6ms/step - loss: 1.9479 - acc: 0.4342
Epoch 6/10
2020/2020 [==============================] - 12s 6ms/step - loss: 1.6199 - acc: 0.5361
Epoch 7/10
2020/2020 [==============================] - 12s 6ms/step - loss: 1.3802 - acc: 0.6030
Epoch 8/10
2020/2020 [==============================] - 11s 6ms/step - loss: 1.1496 - acc: 0.6668
Epoch 9/10
2020/2020 [==============================] - 12s 6ms/step - loss: 1.0077 - acc: 0.7124
Epoch 10/10
2020/2020 [==========

Predicted emotion:  Fear
Actual emotion   :  Anger



Oh will you give it a rest.

Predicted emotion:  Anger
Actual emotion   :  Anger



Now though, the gloves are off.

Predicted emotion:  Anticipation
Actual emotion   :  Aggression



Just as he will not tolerate questions about his private life, so he will also go ballistic if bureacrats attempt to meddle in his creative decisions.

Predicted emotion:  Anticipation
Actual emotion   :  Anger



Now don't go off the deep end.

Predicted emotion:  Anticipation
Actual emotion   :  Anticipation



Old Stevenson would go spare if he knew.

Predicted emotion:  Sadness
Actual emotion   :  Neutral



Yeah, it's good riddance, probably.

Predicted emotion:  Contempt
Actual emotion   :  Remorse



Vegetarianism is purely a matter of choice and there's no people that harp on about you shouldn't eat meat.

Predicted emotion:  Anger
Actual emotion   :  Ambiguous



Some got extremely hot under the collar about it.

Predicted emotion:  Anger
Actu


Predicted emotion:  Neutral
Actual emotion   :  Optimism



Did you have a lump in your throat?

Predicted emotion:  Fear
Actual emotion   :  Fear



I have very mixed feelings about it.

Predicted emotion:  Ambiguous
Actual emotion   :  Ambiguous



No hard feelings, darling, so nice you're getting married?

Predicted emotion:  Sadness
Actual emotion   :  Optimism



Since immigrants from hot countries have settled here for generations, surviving the cold, icy blast, the hybrid English, with the blood of many nations in their veins will not turn a hair at the cooler climate or the cooler neighbours.

Predicted emotion:  Fear
Actual emotion   :  Neutral



Now that's a quick fix and hasn't cost anything, hasn't put anybody's nose out of joint.

Predicted emotion:  Sadness
Actual emotion   :  Love



Why open old wounds?

Predicted emotion:  Anger
Actual emotion   :  Disapproval



Not only are characters within the tale likened to animals for instance the miller, proud as a peacock, w




This picture of Dreadnought was so wide of the mark that Willis found it better to say nothing, simply making a gesture which had something in common with a petty officer's salute.

Predicted emotion:  Sadness
Actual emotion   :  Neutral



Servants bow and scrape.

Predicted emotion:  Sadness
Actual emotion   :  Submission



Yes, the done thing for driving horses to canter or gallop on the road, they never ever should.

Predicted emotion:  Contempt
Actual emotion   :  Neutral



They didn't switch off when he played bloody hell with' em, excuse my French er President, but this is how it went, he couldn't give a damn about you bloody lot, he kept the television on, it just showed his frustration.

Predicted emotion:  Disapproval
Actual emotion   :  Contempt



Of course, en suite has long been de rigueur, if you'll pardon my French, in hotels nationwide.

Predicted emotion:  Neutral
Actual emotion   :  Ambiguous



Perhaps next time you'll inform me first before you gatecrash my pa




Don't you never look before you leap?

Predicted emotion:  Anticipation
Actual emotion   :  Disapproval



It's a no go area.

Predicted emotion:  Disapproval
Actual emotion   :  Disapproval



To keep the rod upright, out of harm 's way when not in use.

Predicted emotion:  Neutral
Actual emotion   :  Ambiguous



In a recent episode of a Music Television cartoon called Beavis and Butthead, the characters play with fire.

Predicted emotion:  Sadness
Actual emotion   :  Disapproval



It is not in your nature to put your head on the block; so much I do know.

Predicted emotion:  Neutral
Actual emotion   :  Contempt



And would the swans find shelter from the storm, or ride it out like three-masted sailing ships?

Predicted emotion:  Anticipation
Actual emotion   :  Anticipation



Phil Angell asked him what does it take to risk life and limb.

Predicted emotion:  Anticipation
Actual emotion   :  Neutral



Och he knows you're in safe hands like when you come round here.

Predicted 

...




Since immigrants from hot countries have settled here for generations, surviving the cold, icy blast, the hybrid English, with the blood of many nations in their veins will not turn a hair at the cooler climate or the cooler neighbours.

Predicted emotion:  Neutral
Actual emotion   :  Neutral



Now that's a quick fix and hasn't cost anything, hasn't put anybody's nose out of joint.

Predicted emotion:  Neutral
Actual emotion   :  Love



Why open old wounds?

Predicted emotion:  Love
Actual emotion   :  Disapproval



Not only are characters within the tale likened to animals for instance the miller, proud as a peacock, with his skull.

Predicted emotion:  Neutral
Actual emotion   :  Neutral



Yeah, pleased as punch.

Predicted emotion:  Anticipation
Actual emotion   :  Joy



Time to plan a dumpbin and regain my composure.

Predicted emotion:  Sadness
Actual emotion   :  Ambiguous



But really, what, all they wanted to do was save face from the hospital because it wasn't our fau


Oh, just thinking, I'm not all there.

Predicted emotion:  Fear
Actual emotion   :  Ambiguous



It may very well be that I'm not playing with a full deck.

Predicted emotion:  Optimism
Actual emotion   :  Sadness



If you think about it long enough you'll go nuts.

Predicted emotion:  Disapproval
Actual emotion   :  Anticipation



You're off your rocker.

Predicted emotion:  Fear
Actual emotion   :  Disapproval



I swear you have a screw loose.

Predicted emotion:  Anger
Actual emotion   :  Disgust



I must have gone stark raving mad.

Predicted emotion:  Neutral
Actual emotion   :  Remorse



I'm gonna get stir crazy not being able to play badminton I know, and I score, it's gonna drive me crazy.

Predicted emotion:  Disapproval
Actual emotion   :  Sadness



It may well be necessary to botch up repairs using hurdles, bedsteads, corrugated iron sheets, scraps of wire netting, and baler twine.

Predicted emotion:  Neutral
Actual emotion   :  Neutral



It is now time, however, th

Predicted emotion:  Neutral
Actual emotion   :  Neutral



Treasurers suffer the erm, the occupational hazard that they always, to some extent, have to cry wolf.

Predicted emotion:  Love
Actual emotion   :  Optimism



Hoomey's teeth were chattering after his dice with death.

Predicted emotion:  Neutral
Actual emotion   :  Fear



There is a legal case pending, and it is a very, very dicey situation.

Predicted emotion:  Remorse
Actual emotion   :  Fear



And with marks to prove it, that er just sort of one false move and my job with life was over.

Predicted emotion:  Neutral
Actual emotion   :  Neutral



The whole area seemed fraught with danger.

Predicted emotion:  Disapproval
Actual emotion   :  Fear



You escaped death by a hair 's breadth.

Predicted emotion:  Neutral
Actual emotion   :  Anticipation



Obey them to the letter and you will be well rewarded -- any bungling or playing the fool and I doubt you'll live to tell the tale

Predicted emotion:  Contempt
Actual emoti

Predicted emotion:  Neutral
Actual emotion   :  Disapproval



Whereas before we were going up in leaps and bounds.

Predicted emotion:  Sadness
Actual emotion   :  Ambiguous



I reluctantly withdraw this motion, but I will say this, we live to fight another day.

Predicted emotion:  Neutral
Actual emotion   :  Remorse



Joe and I mean I come a long way

Predicted emotion:  Neutral
Actual emotion   :  Love



If you find yourself at a low ebb this weekend don't worry.

Predicted emotion:  Submission
Actual emotion   :  Awe



The station which persuaded him to make a comeback says it can't believe its luck.

Predicted emotion:  Neutral
Actual emotion   :  Joy



Why did you make the cut worse?

Predicted emotion:  Neutral
Actual emotion   :  Disapproval



They therefore faced no option other than to make a go of the business.

Predicted emotion:  Submission
Actual emotion   :  Neutral



It was a bit like the rowing boat trying to make headway against the flow of the river near the 




I might be wrong, but I don't think so, but I have to sit on the fence you see as I get into trouble.

Predicted emotion:  Neutral
Actual emotion   :  Neutral



Don't split hairs.

Predicted emotion:  Neutral
Actual emotion   :  Trust



That was a long time ago, there's been a lot of water under the bridge, why not talk again?

Predicted emotion:  Optimism
Actual emotion   :  Optimism



Having won already this season on the main tour  he believes he has it in him to do so again, though not to wipe the slate clean after his defeat in a play-off against Nick Faldo in 1990.

Predicted emotion:  Ambiguous
Actual emotion   :  Optimism



The ad-man opposite had been casting meaningful glances at her all evening, and the likeliest explanation seemed to be that she and Roger were doing a number together and I'd inadvertently got caught in the crossfire.

Predicted emotion:  Disapproval
Actual emotion   :  Surprise



They wondered why it was that the storm had left them in this stifling

In [9]:
from numpy import delete, zeros

sums = zeros(18, dtype=int)

for i in range(len(mistake_list)):
    for j in range(len(mistake_list)):
        mistake_list[i][j] = mistake_list[i][j] * 100 / list(emotion_list).count(unique(emotion_list)[i]) 

print(mistake_list)

for i in range(len(mistake_list)):
    sums[i] = sum(mistake_list[i])


strings = []
for i in range(len(mistake_list)):
    temp_list = mistake_list[i]
    strings.append("\\textbf{")
    strings[i] +=  unique(emotion_list)[i] + "}&"
    for j in range(5):
        strings[i] += unique(emotion_list)[list(temp_list).index(max(temp_list))] + " & "  + str(max(temp_list)) + "& "
        temp_list[list(temp_list).index(max(temp_list))] = 0
    strings[i] += "\\\ "
print("hello")
h = "".join(strings)
print(h)

[[ 0  2  4  9  0  6  9  0  0  2  0 22  9  4  6  0  4  0]
 [ 0  0  7  6  0  6 17  1  6  8  0 38 11  0  4  3  5  1]
 [ 1  3  0  2  0  5 10  1  7  5  2 27  6  0  5  1  2  0]
 [ 0  2  4  0  0  1  5  0  4  7  1 14  8  0  1  0  1  0]
 [ 0  6  3  6  0  0  3  0  3  3  0 25  6  0  3  3  0  0]
 [ 0  3  6  7  0  0 13  1  7  4  0 27  4  1  2  0  2  0]
 [ 0  2  7  6  0  2  0  0  4  5  2 22 11  0  4  2  3  0]
 [ 2  2  6  2  0  2  9  0  4  9  2 23 16  0  6  0  4  0]
 [ 0  3  6  4  0  1 13  0  0  1  3 17  3  0  0  0  0  0]
 [ 1  2  5  4  0  1  8  0  3  0  1 20  5  0  0  2  2  0]
 [ 0  4  4  4  0  2 11  0  5  2  0 11 10  0  2  1  2  0]
 [ 0  2  6  5  0  4  7  0  5  3  1  0  7  0  2  2  2  0]
 [ 0  2  2  6  0  4  9  0  2  6  2 21  0  0  2  0  1  0]
 [ 0 11  6  9  0  4  6  2  0  2  0 31  6  0  6  2  0  0]
 [ 1  3  6  4  0  2 10  0  3  2  5 23 11  0  0  2  0  0]
 [ 0  3  6  5  0  3  3  0  6  6  1 16  2  0  2  0  1  0]
 [ 0  0  5  6  0  0 11  0  1  5  0 21  7  0  3  1  0  0]
 [ 3  0  0  3  0  3  0  0  0  3